In [1]:
import os
import sys

In [2]:
sys.path.append(os.path.expanduser("~/unity-py"))

In [3]:
from unity_py.services.application_service import *

## Configuration

In [4]:
dockstore_api_uri = "http://uads-test-dockstore-deploy-lb-1762603872.us-west-2.elb.amazonaws.com:9998/api"

In [5]:
token = !cat ~/.dockstore/config  | grep token | awk '{print $2}'
token = token[0]

## Catalog Interface


In [6]:
app_catalog = DockstoreAppCatalog(dockstore_api_uri, token)

## Existing User Applications

In [7]:
for app in app_catalog.application_list(for_user=True):
    print(f"{app.id} {app.name} {app.source_repository}")

## New Application

In [8]:
name = "sounder_sips_l1a"
source_repository = "git@github.com:unity-sds/sounder-sips-application.git"
workflow_path = "/cwl/l1a_workflow.cwl"

In [9]:
new_app = ApplicationPackage(name, source_repository, workflow_path)

In [10]:
new_app

ApplicationPackage(name='sounder_sips_l1a', source_repository='git@github.com:unity-sds/sounder-sips-application.git', workflow_path='/cwl/l1a_workflow.cwl', id=None, is_published=False, description='', workflow_type='CWL')

## Publish Application

In [11]:
reg_app = app_catalog.register(new_app)

In [12]:
reg_app

DockstoreApplicationPackage(name='sounder_sips_l1a', source_repository='git@github.com:unity-sds/sounder-sips-application.git', workflow_path='/cwl/l1a_workflow.cwl', id='50', is_published=False, description='Processes Sounder SIPS L0 products into L1A products', workflow_type='CWL', dockstore_info={'type': 'BioWorkflow', 'descriptorType': 'CWL', 'aliases': {}, 'author': 'James McDuffie', 'checker_id': None, 'conceptDoi': None, 'dbCreateDate': 1670623272833, 'dbUpdateDate': 1670623276387, 'defaultTestParameterFilePath': '/test.json', 'defaultVersion': 'main', 'description': 'Processes Sounder SIPS L0 products into L1A products', 'descriptorTypeSubclass': 'NOT_APPLICABLE', 'email': 'James.Mcduffie@jpl.nasa.gov', 'forumUrl': None, 'full_workflow_path': 'github.com/unity-sds/sounder-sips-application/sounder_sips_l1a', 'gitUrl': 'git@github.com:unity-sds/sounder-sips-application.git', 'has_checker': False, 'id': 50, 'input_file_formats': [], 'isChecker': False, 'is_published': False, 'labe

## Delete Application (Cleanup)

In [13]:
del_app = app_catalog.unregister(reg_app, delete=True)

In [14]:
del_app

DockstoreApplicationPackage(name='sounder_sips_l1a', source_repository='git@github.com:unity-sds/sounder-sips-application.git', workflow_path='/cwl/l1a_workflow.cwl', id=None, is_published=False, description='Processes Sounder SIPS L0 products into L1A products', workflow_type='CWL', dockstore_info=None)